In [ ]:
# 在 notebook 中添加调试代码：
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

# 尝试训练
!python tools/train.py -c configs/rtdetr/rtdetr_r50vd_6x_hrsid.yml

^C


: 